<a href="https://colab.research.google.com/github/Hemashree2407/SQL_Project1/blob/main/New%20copy%20part%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Indian Equity Analyzer - Complete Version Part 1
# This is the FIRST PART of the complete script including all bug fixes and enhancements

import yfinance as yf
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple, Union
from dataclasses import dataclass
import json
import warnings
import requests
from bs4 import BeautifulSoup
import concurrent.futures
from functools import lru_cache

warnings.filterwarnings('ignore')

# ===========================
# DATA CLASSES AND STRUCTURES
# ===========================

@dataclass
class StockScore:
    """Data class to hold stock analysis scores and metrics"""
    symbol: str
    fundamental_score: float
    technical_score: float
    sentiment_score: float
    composite_score: float
    analysis_date: str
    key_metrics: Dict
    data_quality_score: float
    peer_comparison_score: float
    liquidity_score: float
    index_membership: List[str]

# ===========================
# DATA SOURCE MANAGER
# ===========================

class DataSourceManager:
    """Manages multiple data sources with fallback mechanism"""

    def __init__(self):
        self.sources = {
            'yfinance': self._get_yfinance_data,
            'nsepy': self._get_nsepy_data,
            'moneycontrol': self._get_moneycontrol_data
        }
        self.cache = {}
        self.cache_expiry = 300  # 5 minutes

    def get_consolidated_data(self, symbol: str, source_priority: List[str] = None) -> Optional[Dict]:
        """Try multiple sources with fallback"""
        if source_priority is None:
            source_priority = ['yfinance', 'nsepy', 'moneycontrol']

        # Check cache first
        cache_key = f"{symbol}_data"
        if cache_key in self.cache:
            cached_data, timestamp = self.cache[cache_key]
            if (datetime.now() - timestamp).seconds < self.cache_expiry:
                return cached_data

        # Try each source in priority order
        for source in source_priority:
            if source in self.sources:
                try:
                    data = self.sources[source](symbol)
                    if data and self._validate_data(data):
                        # Cache the data
                        self.cache[cache_key] = (data, datetime.now())
                        return data
                except Exception as e:
                    print(f"Error fetching from {source}: {str(e)}")
                    continue

        return None

    def _get_yfinance_data(self, symbol: str) -> Dict:
        """Get data from Yahoo Finance"""
        stock = yf.Ticker(symbol)

        return {
            'price_data': stock.history(period="1y"),
            'info': stock.info,
            'balance_sheet': stock.balance_sheet,
            'income_statement': stock.financials,
            'cash_flow': stock.cashflow,
            'quarterly_financials': stock.quarterly_financials,
            'quarterly_balance_sheet': stock.quarterly_balance_sheet,
            'actions': stock.actions,
            'source': 'yfinance'
        }

    def _get_nsepy_data(self, symbol: str) -> Dict:
        """Get data from NSEPy (placeholder for actual implementation)"""
        # This would require nsepy library installation and implementation
        # For now, returning None to fallback to yfinance
        return None

    def _get_moneycontrol_data(self, symbol: str) -> Dict:
        """Get additional data from MoneyControl (placeholder)"""
        # This would involve web scraping MoneyControl
        # For now, returning None to fallback to yfinance
        return None

    def _validate_data(self, data: Dict) -> bool:
        """Validate that essential data is present"""
        required_keys = ['price_data', 'info', 'balance_sheet', 'income_statement']
        return all(key in data and data[key] is not None for key in required_keys)

# ===========================
# MARKET DATA FETCHER
# ===========================

class MarketDataFetcher:
    """Fetches market-specific data like risk-free rate, index constituents, etc."""

    def __init__(self):
        self.cache = {}
        self.cache_expiry = 3600  # 1 hour

    @lru_cache(maxsize=1)
    def get_current_risk_free_rate(self) -> float:
        """Fetch current 10-year G-Sec yield dynamically"""
        try:
            # Try to get from World Government Bonds
            response = requests.get(
                "https://www.worldgovernmentbonds.com/country/india/",
                headers={'User-Agent': 'Mozilla/5.0'}
            )
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                # Parse the 10-year yield (implementation depends on website structure)
                # For now, using a recent realistic value
                return 0.072  # 7.2% as of recent data
        except:
            pass

        # Fallback to default
        return 0.065

    def get_index_constituents(self, index_name: str) -> List[str]:
        """Get constituents of major indices"""
        index_mapping = {
            'NIFTY50': self._get_nifty50_constituents,
            'NIFTY_NEXT50': self._get_next50_constituents,
            'NIFTY_MIDCAP': self._get_midcap_constituents,
            'SENSEX': self._get_sensex_constituents
        }

        if index_name in index_mapping:
            return index_mapping[index_name]()
        return []

    def _get_nifty50_constituents(self) -> List[str]:
        """Get NIFTY 50 constituents"""
        # In production, this would fetch from NSE website
        # For now, returning top stocks
        return [
            'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS',
            'HDFC.NS', 'SBIN.NS', 'BHARTIARTL.NS', 'KOTAKBANK.NS', 'ITC.NS',
            'LT.NS', 'AXISBANK.NS', 'BAJFINANCE.NS', 'MARUTI.NS', 'ASIANPAINT.NS',
            'HINDUNILVR.NS', 'TITAN.NS', 'SUNPHARMA.NS', 'NESTLEIND.NS', 'WIPRO.NS',
            'ULTRACEMCO.NS', 'TECHM.NS', 'HCLTECH.NS', 'POWERGRID.NS', 'NTPC.NS',
            'TATAMOTORS.NS', 'INDUSINDBK.NS', 'TATASTEEL.NS', 'ADANIPORTS.NS', 'ONGC.NS'
        ]

    def _get_next50_constituents(self) -> List[str]:
        """Get NIFTY Next 50 constituents"""
        return [
            'ADANIGREEN.NS', 'ADANITRANS.NS', 'AMBUJACEM.NS', 'APOLLOHOSP.NS',
            'BANDHANBNK.NS', 'BANKBARODA.NS', 'BERGEPAINT.NS', 'BIOCON.NS',
            'BOSCHLTD.NS', 'CADILAHC.NS', 'CHOLAFIN.NS', 'COALINDIA.NS'
        ]

    def _get_midcap_constituents(self) -> List[str]:
        """Get NIFTY Midcap constituents"""
        return [
            'AARTIIND.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS',
            'AJANTPHARM.NS', 'ALKEM.NS', 'AMARAJABAT.NS', 'APOLLOTYRE.NS',
            'ASHOKLEY.NS', 'ASTRAZEN.NS', 'ATUL.NS', 'AUBANK.NS'
        ]

    def _get_sensex_constituents(self) -> List[str]:
        """Get SENSEX constituents"""
        return [
            'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'ICICIBANK.NS',
            'HDFC.NS', 'ITC.NS', 'SBIN.NS', 'BHARTIARTL.NS', 'KOTAKBANK.NS',
            'LT.NS', 'AXISBANK.NS', 'WIPRO.NS', 'ULTRACEMCO.NS', 'SUNPHARMA.NS',
            'MARUTI.NS', 'TITAN.NS', 'NESTLEIND.NS', 'HINDUNILVR.NS', 'ASIANPAINT.NS'
        ]

# ===========================
# MAIN EQUITY ANALYZER CLASS
# ===========================

class EquityAnalyzer:
    """Main class for comprehensive equity analysis of Indian stocks"""

    def __init__(self, db_path: str = "indian_equity_analysis.db"):
        self.db_path = db_path
        self.data_source_manager = DataSourceManager()
        self.market_data_fetcher = MarketDataFetcher()
        self.setup_database()
        self._initialize_industry_benchmarks()

    def setup_database(self):
        """Initialize SQLite database for storing analysis results"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS stock_analysis (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                symbol TEXT,
                analysis_date TEXT,
                fundamental_score REAL,
                technical_score REAL,
                sentiment_score REAL,
                composite_score REAL,
                data_quality_score REAL,
                peer_comparison_score REAL,
                liquidity_score REAL,
                index_membership TEXT,
                key_metrics TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        conn.commit()
        conn.close()

    def _initialize_industry_benchmarks(self):
        """Initialize comprehensive industry benchmarks with sub-industries for Indian markets"""
        self.INDUSTRY_BENCHMARKS = {
            'technology': {
                'it_services': {
                    'net_profit_margin': {'excellent': 0.22, 'good': 0.16, 'fair': 0.12},
                    'roe': {'excellent': 0.28, 'good': 0.20, 'fair': 0.15},
                    'roce': {'excellent': 0.35, 'good': 0.25, 'fair': 0.18},
                    'ebitda_margin': {'excellent': 0.25, 'good': 0.18, 'fair': 0.10},
                    'debt_equity': {'excellent': 0.2, 'good': 0.4, 'fair': 0.6},
                    'current_ratio': {'excellent': 2.0, 'good': 1.5, 'fair': 1.2}
                },
                'software_products': {
                    'net_profit_margin': {'excellent': 0.25, 'good': 0.18, 'fair': 0.12},
                    'roe': {'excellent': 0.30, 'good': 0.22, 'fair': 0.15},
                    'r_and_d_ratio': {'excellent': 0.15, 'good': 0.10, 'fair': 0.05},
                }
            },
            'financial': {
                'private_banks': {
                    'return_on_assets': {'excellent': 0.018, 'good': 0.015, 'fair': 0.012},
                    'net_interest_margin': {'excellent': 0.045, 'good': 0.038, 'fair': 0.032},
                    'gross_npa_ratio': {'excellent': 0.02, 'good': 0.04, 'fair': 0.06},
                    'capital_adequacy_ratio': {'excellent': 0.16, 'good': 0.14, 'fair': 0.12},
                    'cost_to_income_ratio': {'excellent': 0.40, 'good': 0.45, 'fair': 0.55}
                },
                'public_banks': {
                    'return_on_assets': {'excellent': 0.012, 'good': 0.008, 'fair': 0.005},
                    'net_interest_margin': {'excellent': 0.035, 'good': 0.028, 'fair': 0.022},
                    'gross_npa_ratio': {'excellent': 0.04, 'good': 0.06, 'fair': 0.08},
                },
                'nbfc': {
                    'return_on_assets': {'excellent': 0.025, 'good': 0.018, 'fair': 0.012},
                    'net_interest_margin': {'excellent': 0.055, 'good': 0.045, 'fair': 0.035},
                }
            },
            'pharmaceutical': {
                'generic': {
                    'net_profit_margin': {'excellent': 0.12, 'good': 0.08, 'fair': 0.05},
                    'ebitda_margin': {'excellent': 0.18, 'good': 0.14, 'fair': 0.10},
                    'roe': {'excellent': 0.18, 'good': 0.14, 'fair': 0.10},
                    'debt_equity': {'excellent': 0.3, 'good': 0.5, 'fair': 0.8}
                },
                'branded': {
                    'net_profit_margin': {'excellent': 0.18, 'good': 0.14, 'fair': 0.10},
                    'ebitda_margin': {'excellent': 0.25, 'good': 0.20, 'fair': 0.15},
                    'marketing_to_revenue': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                },
                'api': {
                    'net_profit_margin': {'excellent': 0.15, 'good': 0.10, 'fair': 0.06},
                    'asset_turnover': {'excellent': 1.2, 'good': 0.9, 'fair': 0.6},
                }
            },
            'manufacturing': {
                'auto_oem': {
                    'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.04},
                    'inventory_turnover': {'excellent': 12, 'good': 9, 'fair': 6},
                    'roce': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10},
                    'debt_equity': {'excellent': 0.5, 'good': 0.8, 'fair': 1.2}
                },
                'auto_ancillary': {
                    'net_profit_margin': {'excellent': 0.08, 'good': 0.06, 'fair': 0.04},
                    'working_capital_days': {'excellent': 60, 'good': 90, 'fair': 120},
                },
                'capital_goods': {
                    'order_book_to_revenue': {'excellent': 2.5, 'good': 2.0, 'fair': 1.5},
                    'roce': {'excellent': 0.18, 'good': 0.14, 'fair': 0.10},
                }
            },
            'fmcg': {
                'food_beverages': {
                    'net_profit_margin': {'excellent': 0.12, 'good': 0.09, 'fair': 0.06},
                    'inventory_turnover': {'excellent': 15, 'good': 12, 'fair': 9},
                    'roe': {'excellent': 0.30, 'good': 0.20, 'fair': 0.15}
                },
                'personal_care': {
                    'net_profit_margin': {'excellent': 0.15, 'good': 0.12, 'fair': 0.08},
                    'advertising_to_revenue': {'excellent': 0.12, 'good': 0.09, 'fair': 0.06},
                }
            },
            'energy': {
                'oil_gas': {
                    'net_profit_margin': {'excellent': 0.10, 'good': 0.07, 'fair': 0.04},
                    'reserve_replacement_ratio': {'excellent': 1.2, 'good': 1.0, 'fair': 0.8},
                    'debt_equity': {'excellent': 0.8, 'good': 1.2, 'fair': 1.8}
                },
                'power': {
                    'plant_load_factor': {'excellent': 0.85, 'good': 0.75, 'fair': 0.65},
                    'debt_equity': {'excellent': 1.5, 'good': 2.0, 'fair': 2.5},
                }
            },
            'real_estate': {
                'residential': {
                    'pre_sales_to_revenue': {'excellent': 1.5, 'good': 1.2, 'fair': 1.0},
                    'debt_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0},
                },
                'commercial': {
                    'occupancy_rate': {'excellent': 0.90, 'good': 0.80, 'fair': 0.70},
                    'rental_yield': {'excellent': 0.08, 'good': 0.06, 'fair': 0.04},
                }
            },
            'telecom': {
                'arpu': {'excellent': 200, 'good': 150, 'fair': 100},
                'subscriber_churn': {'excellent': 0.02, 'good': 0.03, 'fair': 0.05},
                'ebitda_margin': {'excellent': 0.40, 'good': 0.32, 'fair': 0.25},
                'debt_equity': {'excellent': 1.0, 'good': 1.5, 'fair': 2.0}
            },
            'healthcare': {
                'hospitals': {
                    'bed_occupancy_rate': {'excellent': 0.80, 'good': 0.70, 'fair': 0.60},
                    'arpob': {'excellent': 50000, 'good': 40000, 'fair': 30000},
                    'ebitda_margin': {'excellent': 0.20, 'good': 0.15, 'fair': 0.10}
                },
                'diagnostics': {
                    'revenue_per_test': {'excellent': 800, 'good': 600, 'fair': 400},
                    'ebitda_margin': {'excellent': 0.25, 'good': 0.20, 'fair': 0.15},
                }
            }
        }

    def get_stock_data(self, symbol: str, period: str = "1y") -> Optional[Dict[str, any]]:
        """Fetch comprehensive stock data from multiple sources with fallback"""
        # Use data source manager for better reliability
        data = self.data_source_manager.get_consolidated_data(symbol)

        if not data:
            print(f"❌ Failed to fetch data for {symbol} from any source")
            return None

        # Add current risk-free rate
        data['risk_free_rate'] = self.market_data_fetcher.get_current_risk_free_rate()

        # Check index membership
        data['index_membership'] = self._check_index_membership(symbol)

        # Verify and adjust for corporate actions
        data = self._verify_corporate_actions(symbol, data)

        return data

    def _check_index_membership(self, symbol: str) -> List[str]:
        """Check which major indices the stock belongs to"""
        indices = []

        index_checks = {
            'NIFTY50': self.market_data_fetcher.get_index_constituents('NIFTY50'),
            'NIFTY_NEXT50': self.market_data_fetcher.get_index_constituents('NIFTY_NEXT50'),
            'NIFTY_MIDCAP': self.market_data_fetcher.get_index_constituents('NIFTY_MIDCAP'),
            'SENSEX': self.market_data_fetcher.get_index_constituents('SENSEX')
        }

        for index_name, constituents in index_checks.items():
            if symbol in constituents:
                indices.append(index_name)

        return indices

    def _verify_corporate_actions(self, symbol: str, data: Dict) -> Dict:
        """Verify and adjust for recent corporate actions"""
        try:
            actions = data.get('actions', pd.DataFrame())
            if not actions.empty:
                # Check for recent actions (last 90 days)
                recent_date = datetime.now() - timedelta(days=90)
                recent_actions = actions[actions.index > recent_date]

                if not recent_actions.empty:
                    print(f"⚠️ Recent corporate actions detected for {symbol}:")
                    for idx, row in recent_actions.iterrows():
                        if row.get('Stock Splits', 0) > 0:
                            print(f"  - Stock Split: {row['Stock Splits']} on {idx.date()}")
                        if row.get('Dividends', 0) > 0:
                            print(f"  - Dividend: ₹{row['Dividends']} on {idx.date()}")

                    # Add flag to data
                    data['has_recent_corporate_actions'] = True
                    data['recent_actions'] = recent_actions.to_dict()
        except Exception as e:
            print(f"Error checking corporate actions: {str(e)}")

        return data

    def classify_industry(self, info: Dict) -> Tuple[str, str]:
        """Enhanced industry classification with sub-industries for Indian markets"""
        sector = info.get('sector', '').lower()
        industry = info.get('industry', '').lower()
        company_name = info.get('longName', '').lower()

        # Financial services
        if any(keyword in sector or keyword in industry for keyword in ['bank', 'financ']):
            if 'private' in industry or any(bank in company_name
                                          for bank in ['hdfc', 'icici', 'axis', 'kotak', 'yes bank']):
                return 'financial', 'private_banks'
            elif 'public' in industry or any(bank in company_name
                                            for bank in ['sbi', 'state bank', 'bank of baroda', 'pnb', 'canara', 'union bank']):
                return 'financial', 'public_banks'
            elif 'nbfc' in industry or any(nbfc in company_name
                                          for nbfc in ['bajaj', 'mahindra', 'muthoot', 'shriram']):
                return 'financial', 'nbfc'
            return 'financial', 'general'

        # Technology
        elif any(keyword in sector or keyword in industry for keyword in ['technology', 'software', 'information tech']):
            if 'services' in industry or any(company in company_name
                                            for company in ['infosys', 'tcs', 'wipro', 'hcl tech', 'tech mahindra']):
                return 'technology', 'it_services'
            elif 'product' in industry:
                return 'technology', 'software_products'
            return 'technology', 'general'

        # Pharmaceutical
        elif any(keyword in sector or keyword in industry for keyword in ['pharma', 'drug', 'healthcare']):
            if any(keyword in company_name for keyword in ['generic', 'cipla', 'sun pharma', 'lupin', 'dr reddy']):
                return 'pharmaceutical', 'generic'
            elif any(keyword in company_name for keyword in ['abbott', 'glaxo', 'pfizer', 'sanofi']):
                return 'pharmaceutical', 'branded'
            elif 'api' in company_name or 'bulk drug' in industry:
                return 'pharmaceutical', 'api'
            return 'pharmaceutical', 'general'

        # Manufacturing
        elif any(keyword in sector or keyword in industry for keyword in ['auto', 'manufacturing', 'industrial']):
            if 'automobile' in industry or any(company in company_name
                                             for company in ['maruti', 'tata motors', 'mahindra', 'bajaj auto', 'hero']):
                return 'manufacturing', 'auto_oem'
            elif 'auto component' in industry or 'ancillary' in industry:
                return 'manufacturing', 'auto_ancillary'
            elif 'capital goods' in industry:
                return 'manufacturing', 'capital_goods'
            return 'manufacturing', 'general'

        # FMCG
        elif any(keyword in sector or keyword in industry for keyword in ['consumer', 'fmcg']):
            if 'food' in industry or 'beverage' in industry:
                return 'fmcg', 'food_beverages'
            elif 'personal care' in industry:
                return 'fmcg', 'personal_care'
            return 'fmcg', 'general'

        # Energy
        elif any(keyword in sector or keyword in industry for keyword in ['energy', 'oil', 'gas', 'power']):
            if 'oil' in industry or 'gas' in industry:
                return 'energy', 'oil_gas'
            elif 'power' in industry or 'electricity' in industry:
                return 'energy', 'power'
            return 'energy', 'general'

        # Real Estate
        elif any(keyword in sector or keyword in industry for keyword in ['real estate', 'realty']):
            if 'residential' in company_name:
                return 'real_estate', 'residential'
            elif 'commercial' in company_name:
                return 'real_estate', 'commercial'
            return 'real_estate', 'general'

        # Telecom
        elif any(keyword in sector or keyword in industry for keyword in ['telecom', 'communication']):
            return 'telecom', 'general'

        # Healthcare
        elif any(keyword in sector or keyword in industry for keyword in ['healthcare', 'hospital', 'diagnostic']):
            if 'hospital' in industry:
                return 'healthcare', 'hospitals'
            elif 'diagnostic' in industry:
                return 'healthcare', 'diagnostics'
            return 'healthcare', 'general'

        else:
            return 'general', 'general'

    # Continue in Part 2...